# Lecture 3.14: Introduction to Pytorch

**This notebook is meant to be run on Google Colab. Please start with the other notebook in this directory first if you are running this in jupyter.**

## 5. GPUs

### 5.1 Google Colab

Welcome to google colab! 👋 This is basically a cloud hosted notebook, which can have a GPU runtime for _free_ , courtesy of Google.

First, we need to enable GPUs for the notebook:

- navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Once that's done, let's first check that everything is indeed in place:

In [0]:
import torch

torch.cuda.is_available()

This means that pytorch successfully communicated with [CUDA](https://developer.nvidia.com/cuda-zone), which is Nvidia's GPU api. Thankfully we won't have to implement any of this communication ourselves 😅. We can check the device name:

In [0]:
torch.cuda.get_device_name(0)

To keep GPU runtimes free, Google colab doesn't guarantee which hardware is provided to our notebook. So it's good to check which kind we got!

Next, we want to choose the GPU as pytorch device. This is done as follows:

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

The returned device confirms that torch is using the GPU. 

### 5.2 Using GPUs with pytorch

Running matrix calculations on GPU is easy with pytorch. All we have to do is to _move_ the relevant tensors to the GPU `device`. When tensor memory allocation is moved to a GPU, pytorch takes care of all the low level machinery to parallelize and speed up computation.

Before we do that however, let's reload our dataset and build our neural network here in Google colab. Since this notebook running in the cloud, its runtime doesn't have access to local files, including the `.csv` datasets in our local repository. However, these are hosted on GitHub, so we can download them to this notebook's temporary file system with [wget](https://www.gnu.org/software/wget/):

In [0]:
!wget https://raw.githubusercontent.com/camille-vanhoffelen/introduction-to-machine-learning/master/data_analysis/lecture3.14/bank_note.csv

We can now load our dataset as per usual with pandas:

In [0]:
import pandas as pd

df = pd.read_csv('bank_note.csv')
df.head()

We can then prepare the dataset, and create the neural network. This is still exactly the same code as the other 3.14 notebook:

In [0]:
from torch.utils.data import DataLoader

X = df[['feature_1', 'feature_2', 'feature_3', 'feature_4']].values
y = df['is_fake'].values

dataset = list(zip(X, y))
dataset[0]

ds_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [0]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # create the layers
        self.dense1 = nn.Linear(4, 6)
        self.dense2 = nn.Linear(6, 6)
        self.dense3 = nn.Linear(6, 1)

    def forward(self, x):
        # first hidden layer
        x = F.relu(self.dense1(x))
        # second hidden layer
        x = F.relu(self.dense2(x))
        # output layer
        x = torch.sigmoid(self.dense3(x))
        return x

We are ready to train our neural network. Two extra steps are needed compared to CPU training:
- sending the model parameters to GPU device
- sending the features and labels to GPU device at every gradient descent step

Once the memory allocation of these `Tensor`s is moved to the GPU, pytorch understands which operations need to be executed there, and takes care of the rest 🤗.

Let's initialize our neural net and send the $\theta$s to the GPU:

In [0]:
net = Net()
net.to(device)

That's it! Now we can run our training loop, but we'll also have to use the `.to(device)` method on the `inputs` and `labels` variable each iteration:

In [0]:
import numpy as np

# reproducibility
torch.manual_seed(1337)
np.random.seed(666)

# initialization
net = Net()
net.to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

losses = []

# loop over epochs
for epoch in range(100):
    print(f'epoch {epoch} ')
    
    # loop over batches
    for i, data in enumerate(ds_loader):
        # data loading
        inputs, labels = data
        inputs = inputs.float().to(device)
        labels = labels.float().view(-1, 1).to(device)
        
        # prediction
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        # optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics
        losses.append(loss.item())

print('finished Training')

That wasn't much faster than our local CPU training 😕 This is because GPU acceleration for neural networks is complicated. Not all operations are parallelised in the same way, so some layer types and widths benefit more than others. Typically, the speed up becomes obvious for large computer vision and natural language processing models. But more on that in the next lectures! 

In the meantime, let's check that our neural network was properly trained. We can use matplotlib to plot the loss curve in Google colab too:

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()


fig = plt.figure(dpi=100)
ax = fig.add_subplot(111)
ax.plot(losses, lw=1)
ax.set_xlabel('batch')
ax.set_ylabel('steps')
ax.set_title('Loss Curve');

Looks great! Congratulations on training your first neural network on a GPU 🎊. These banknote counterfeiters better behave!

Let's move back to the other 3.14 local jupyter notebook to conclude the lecture.